# **Setup**

In [1]:
!python3 --version

from psutil import *
print('CPU Core Count: ',cpu_count())
print(f'System RAM: {virtual_memory()[0]/1024/1024/1024:.2f} GB')

Python 3.11.4
CPU Core Count:  8
System RAM: 8.00 GB


In [2]:
import pandas as pd 
import numpy as np
import plotly.express as px 

# **Data Import**

In [3]:
df = pd.read_csv('statehood_data.csv')

print(df.shape)
print(df.columns)

(50, 4)
Index(['state', 'abbr', 'date_entered', 'order'], dtype='object')


In [4]:
df.head()

,state,abbr,date_entered,order
0,Delaware,DE,"Dec. 7, 1787",1
1,Pennsylvania,PA,"Dec. 12, 1787",2
2,New Jersey,NJ,"Dec. 18, 1787",3
3,Georgia,GA,"Jan. 2, 1788",4
4,Connecticut,CT,"Jan. 9, 1788",5


In [5]:
df.isna().sum()

state           0
abbr            0
date_entered    0
order           0
dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   state         50 non-null     object
 1   abbr          50 non-null     object
 2   date_entered  50 non-null     object
 3   order         50 non-null     int64 
dtypes: int64(1), object(3)
memory usage: 1.7+ KB


# **Data Processing**

In [10]:
df['Year'] = df['date_entered'].str[-4:].astype('int')
df.head()

,state,abbr,date_entered,order,Year
0,Delaware,DE,"Dec. 7, 1787",1,1787
1,Pennsylvania,PA,"Dec. 12, 1787",2,1787
2,New Jersey,NJ,"Dec. 18, 1787",3,1787
3,Georgia,GA,"Jan. 2, 1788",4,1788
4,Connecticut,CT,"Jan. 9, 1788",5,1788


# **Create Chloropleth Map**

* Without Animation

In [13]:
fig = px.choropleth(df,locations='abbr',locationmode='USA-states',color='Year',scope='usa',
                    hover_data = ['state','order','Year'],title='State By Date Of Entry Into Union',
                    color_continuous_scale = px.colors.diverging.Portland,)
fig.update_layout(width=750,height=500)

# **Map With Animation**

In [14]:
fig = px.choropleth(df,locations='abbr',locationmode='USA-states',color='Year',scope='usa',
                    hover_data = ['state','order','Year'],title='State By Date Of Entry Into Union',
                    color_continuous_scale = px.colors.diverging.Portland,animation_frame='Year')

fig.update_layout(width=750,height=500)

# **Map With Animation**

In [15]:
# Animate the states grouped by large timeframes:
conditions = [(df['Year'] >= 1787) & (df['Year'] <= 1790),
              (df['Year'] >= 1791) & (df['Year'] <= 1860),
              (df['Year'] >= 1861) & (df['Year'] <= 1900),
              (df['Year'] >= 1901) & (df['Year'] <= 1959)]

values = ['Original 13 Colonies', 'Antebellum Expansion', 
          'Westward Expansion', 'Twentieth Century']

df['Timespan'] = np.select(conditions, values)

In [17]:
fig = px.choropleth(df,locations='abbr',locationmode='USA-states',color='Year',scope='usa',
                    hover_data = ['state','order','Year'],title='State By Date Of Entry Into Union',
                    color_continuous_scale = px.colors.diverging.Portland,animation_frame='Timespan',
                    range_color= (min(df.Year), max(df.Year)))

fig.update_layout(width=750,height=500)

# Set the animation speed (in millseconds):
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 2000
fig.show()

# **Static Frames**

In [18]:
# Make static maps of the states grouped over large timeframes:

# Create new DataFrames for designated time spans
original_13 = df[(df['Year'] >= 1787) & (df['Year'] <= 1790)].copy()
antebellum_expansion = df[(df['Year'] >= 1791) & (df['Year'] <= 1860)].copy()
western_expansion = df[(df['Year'] >= 1861) & (df['Year'] <= 1900)].copy()
twentieth_century = df[(df['Year'] >= 1901) & (df['Year'] <= 1959)].copy()

In [20]:
original_13

,state,abbr,date_entered,order,Year,Timespan
0,Delaware,DE,"Dec. 7, 1787",1,1787,Original 13 Colonies
1,Pennsylvania,PA,"Dec. 12, 1787",2,1787,Original 13 Colonies
2,New Jersey,NJ,"Dec. 18, 1787",3,1787,Original 13 Colonies
3,Georgia,GA,"Jan. 2, 1788",4,1788,Original 13 Colonies
4,Connecticut,CT,"Jan. 9, 1788",5,1788,Original 13 Colonies
5,Massachusetts,MA,"Feb. 6, 1788",6,1788,Original 13 Colonies
6,Maryland,MD,"Apr. 28, 1788",7,1788,Original 13 Colonies
7,South Carolina,SC,"May 23, 1788",8,1788,Original 13 Colonies
8,New Hampshire,NH,"June 21, 1788",9,1788,Original 13 Colonies
9,Virginia,VA,"June 25, 1788",10,1788,Original 13 Colonies


In [19]:
twentieth_century

,state,abbr,date_entered,order,Year,Timespan
45,Oklahoma,OK,"Nov. 16, 1907",46,1907,Twentieth Century
46,New Mexico,NM,"Jan. 6, 1912",47,1912,Twentieth Century
47,Arizona,AZ,"Feb. 14, 1912",48,1912,Twentieth Century
48,Alaska,AK,"Jan. 3, 1959",49,1959,Twentieth Century
49,Hawaii,HI,"Aug. 21, 1959",50,1959,Twentieth Century


In [21]:
# Make a list of DataFrames to loop through
time_frames = [original_13, antebellum_expansion, 
               western_expansion, twentieth_century]

# Make a list of the dates you want to see in the color bar
colorbar_dates = [[1787, 1788, 1789, 1790],
                  [1791, 1800, 1810, 1820, 1830, 1840, 1850, 1859],
                  [1861, 1870, 1880, 1890, 1896],
                  [1907, 1920, 1930, 1940, 1950, 1959]]

In [22]:
# Make a list of the figure titles for each DataFrame
figure_titles = ["Original 13 Colonies (1787-1790)", 
                 "Antebellum Expansion (1791-1860)", 
                 "Western Expansion (1861-1900)", 
                 "Twentieth Century (1901-1959)"]

In [45]:
# Loop through and plot the DataFrames
for i, data_frame in enumerate(time_frames):
    fig = px.choropleth(data_frame,locations='abbr',locationmode='USA-states',color='Year',scope='usa',
                        hover_data = ['state','order'],
                        title=figure_titles[i],
                        color_continuous_scale = px.colors.diverging.Portland,
                        # animation_frame='Timespan',
                        range_color= (min(data_frame['Year']), max(data_frame['Year'])))

    fig.update_layout(width=750,height=500)
    fig.update_layout(coloraxis_colorbar=dict(tickvals=colorbar_dates[i],thickness=15,ticks='outside'))
    fig.show()

# **Static Maps With Persistence**

In [46]:
  # Make static maps of the states grouped over large timeframes, with persistence:

# Create new DataFrames for designated time spans
original_13 = df[(df['Year'] >= 1787) & (df['Year'] <= 1790)].copy()
antebellum_expansion = df[(df['Year'] >= 1787) & (df['Year'] <= 1860)].copy()
western_expansion = df[(df['Year'] >= 1787) & (df['Year'] <= 1900)].copy()
twentieth_century = df[(df['Year'] >= 1787) & (df['Year'] <= 1959)].copy()

# Make a list of DataFrames to loop through
time_frames = [original_13, antebellum_expansion, 
               western_expansion, twentieth_century]

# Make a list of the dates you want to see in the color bar:
colorbar_dates = [1787, 1790, 1800, 1820, 1840, 1859, 
                  1880, 1896, 1910, 1920, 1940, 1959]

# Make a list of the figure titles for each DataFrame
figure_titles = ["States through 1790", 
                 "States through 1860", 
                 "States through 1900", 
                 "States through 1959"]

In [48]:
# Loop through and plot the DataFrames
for i, data_frame in enumerate(time_frames):
    fig = px.choropleth(data_frame,locations='abbr',locationmode='USA-states',color='Year',scope='usa',
                        hover_data = ['state','order'],
                        title=figure_titles[i],
                        color_continuous_scale = px.colors.diverging.Portland,
                        # animation_frame='Timespan',
                        range_color= (min(data_frame['Year']), max(data_frame['Year'])))

    fig.update_layout(width=750,height=500)
    fig.update_layout(coloraxis_colorbar=dict(tickvals=colorbar_dates,thickness=15,ticks='outside'))
    fig.show()